### Example: Copolymer
We setup a system of randomly dispersed `A-A-A-...-A-B-B-...-B-B-B` diblock copolymers, run a short simulation, and examine the resulting trajectory.

In [10]:
import h5py
import numpy as np
from scipy.spatial.transform import Rotation as R


def setup_system(box_size, n_molecules, chain_length, out_path, bond_length=0.5):
    """Setup a simple system of symmetric A/B diblock copolymer chains
    
    Parameters
    ----------
    box_size : list of float
        Simulation box size in nm.
    n_molecules : int
        Total number of molecules of `chain_length` monomer units.
    chain_length : int
        Length of each polymer chain in number of monomers.
    out_path : str
        Path of the created structure file.
    bond_length : float, optional
        Length of intramolecular bonds in the chains.
    """
    box_size = np.asarray(box_size, dtype=np.float32)
    chain_length = int(chain_length - np.mod(chain_length, 2))
    n_half = chain_length // 2
    n_particles = n_molecules * chain_length
    
    with h5py.File(out_path, "w") as out_file:
        position_dataset = out_file.create_dataset(
            "coordinates",
            (1, n_particles, box_size.size,),
            dtype="float32",
        )
        types_dataset = out_file.create_dataset(
            "types",
            (n_particles,),
            dtype="i",
        )
        names_dataset = out_file.create_dataset(
            "names",
            (n_particles,),
            dtype="S10",
        )
        indices_dataset = out_file.create_dataset(
            "indices",
            (n_particles,),
            dtype="i",
        )
        molecules_dataset = out_file.create_dataset(
            "molecules",
            (n_particles,),
            dtype="i",
        )
        bonds_dataset = out_file.create_dataset(
            "bonds",
            (n_particles, 3,),
            dtype="i",
        )
    
        head_positions = np.random.uniform(
            low=np.zeros_like(box_size),
            high=box_size,
            size=(n_molecules, box_size.size,),
        )
        
        # Randomly coiled chains
        for i in range(n_molecules):
            position_dataset[0, i*chain_length, :] = head_positions[i, :]
            for j in range(1, chain_length):
                position_dataset[0, i*chain_length + j, :] = (
                    head_positions[i, :] 
                    + R.random().as_matrix().dot(np.array([
                        bond_length, 0.0, 0.0,
                    ]))
                )
        indices_dataset[...] = np.arange(n_particles)
    
        # molecules
        molecules_dataset[...] = np.repeat(np.arange(n_molecules), chain_length)
        types_dataset[...] = np.tile(
            np.concatenate(
                (
                    np.full(shape=(n_half,), fill_value=0, dtype=int),
                    np.full(shape=(n_half,), fill_value=1, dtype=int),
                ),
            ),
            n_molecules,
        )
        names_dataset[...] = np.tile(
            np.concatenate(
                (
                    np.full(shape=(n_half,), fill_value=np.string_("A")),
                    np.full(shape=(n_half,), fill_value=np.string_("B")),
                ),
            ),
            n_molecules,
        )
            
        # bonds
        bonds_dataset[...] = -1
        bonds_dataset[:, 0] = np.arange(n_particles) - 1
        bonds_dataset[::chain_length, 0] = -1
        
        bonds_dataset[:, 1] = np.arange(n_particles) + 1
        bonds_dataset[chain_length-1::chain_length, 1] = -1
# ==============================================================================

# 416 chains in a 10nm x 10nm x 10nm simulation box.
setup_system([10, 10, 10], 416, 20, "copolymer.HDF5", bond_length=0.25)

### Run the simulation
We the simulation:
```bash
mpirun -n 6 python3 /Users/mortenledum/Documents/HyMD/hymd/__main__.py \
                    copolymer.toml             \
                    copolymer.HDF5             \
                    --seed 1                   \
                    --verbose 1
```